In [0]:
import logging
import requests
from typing import Any
from bs4 import BeautifulSoup

def scrape():
    URL = "https://www.blockchain.com/explorer/mempool/btc"

    transactions = []

    logging.info(f"Scraping website: {URL}")
    try:
        req = requests.get(URL)
    except Exception as e:
        logging.error(f"Failed request. Error:{e}")

    soup = BeautifulSoup(req.text, "html.parser")

    div = soup.find('div', {'class': 'sc-7b53084c-1 czXdjN'})
    a_tags = div.findAll('a')

    for tag in a_tags:
        # hashid
        hash_id = tag.get('href').split("btc/")[1]

        # hash
        hash_tag = tag.find('div', {'class': 'sc-35e7dcf5-5 ozZoF'})
        hash = hash_tag.text.split(' ')[1]

        # date time
        dt = tag.find('div', {'class': 'sc-35e7dcf5-7 fFAyKv'})
        date_time = dt.text.split(', ')
        date = date_time[0]
        time = date_time[1]

        # btc
        btc_tag = tag.find('div', {'class': 'sc-35e7dcf5-13 isCUBz'})
        btc = btc_tag.text.split(' ')[0]
        if ',' in btc:
            btc = btc.replace(',', '')

        # dol
        dol_tag = tag.find('div', {'class': 'sc-35e7dcf5-14 bbQxqN'})
        dol = dol_tag.text[1:]
        if ',' in dol:
            dol = dol.replace(',', '')

        hash_obj = {
            "hash_id": hash_id,
            "hash_short": hash,
            "output_value_btc": float(btc),
            "output_value_dollars": float(dol),
            "date": date,
            "time": time
        }
        transactions.append(hash_obj)
    return transactions

In [0]:
import json
import datetime

checkpoint = datetime.datetime.now().isoformat().split('.')[0]
data = scrape()
json_data = json.dumps(data)
output_path = f"/mnt/transactions/new/{checkpoint}.json"

In [0]:
dbutils.fs.put(output_path, json_data, overwrite=True)
